# Counts statistics in gammapy

## Introduction

### Context

Gamma-ray astronomy is based on individual photon measurements. The existence of a signal is usually
determined by comparing measured number of counts to the expected number of background counts. The latter 
can be known thanks to a model or can be estimated thanks to a second measurement devoid of any signal.

Those two situations drive most of the cases that concern gamma-ray data analysis. The first one is dealt with by the Cash statistic, the second by the WStat.

###Objective

**Compute some statistical informations on the expected number of signal events in a counting experiment. In particular, we want to estimate the statistical significance of the possible signal, the confidence interval or the upper limit on its estimated value.**

### Approach

Counts statistics are handled with dedicated classes in gammapy: `~gammapy.stats.CashCountsStatistic` and
`~gammapy.stats.WStatCountsStatistic`. They are used to estimate the expected signal and the confidence interval of the latter as well as an estimate of the probability that this excess is due to a fluctuation in the number of background counts. This is obtained thanks to a likelihood ratio test and measured as a delta TS in gammapy. This delta TS can be converted to a p-value or to the significance number (often approximated as the square root of the delta TS).  

## Setup

Some imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
from gammapy.stats import CashCountsStatistic, WStatCountsStatistic

## Poisson signal with known Poisson background: Cash statistic

We first study the case of an observation yielding $n_{ON}=13$ events with an expected number of background events of $\mu_{bkg}=5.5$.

We are dealing with Cash statistic and create a `CashCountsStatistic` object.

In [ ]:
count_statistic = CashCountsStatistic(n_on=13, mu_bkg=5.5)

The estimated number of signal events is given by the measured excess and the standard error.

In [ ]:
excess = count_statistic.excess
error = count_statistic.error
print(f"Measured excess : {excess} +- {error}")

### Estimating excess significance

Significance of the measured excess can be estimated by comparing the statistic value for null hypothesis and best fit value. This is the delta TS:

In [ ]:
print(f"Delta TS : {count_statistic.delta_ts}")

According to the Wilks theorem, this delta TS value is asymptotically distributed as a chi-square with 1 degree of freedom. Hence we can have a estimate of the significance, expressed in number of sigma, by taking the square root of delta_ts.

In [ ]:
print(f"Square root TS : {count_statistic.significance}")

Alternatively, we can compute the corresponding p-value: 

In [ ]:
print(f"p_value: {count_statistic.p_value}")

Here, we show how this is computed, plotting the Cash statistic distribution.

In [ ]:
# We compute the Cash statistic profile
mu_signal = np.linspace(-1.5, 25, 100)
stat_values = count_statistic._stat_fcn(mu_signal)

In [ ]:
xmin, xmax = -1.5, 25
ymin, ymax = -42, -28.0
plt.figure(figsize=(7, 7))
plt.plot(mu_signal, stat_values, color="k")
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)

plt.xlabel(r"Number of expected signal event, $\mu_{sig}$")
plt.ylabel(r"Cash statistic value, TS ")
plt.vlines(
    excess,
    ymin=ymin,
    ymax=count_statistic.TS_max,
    linestyle="dashed",
    color="k",
    label="Best fit",
)
plt.hlines(
    count_statistic.TS_max,
    xmin=xmin,
    xmax=excess,
    linestyle="dashed",
    color="k",
)
plt.hlines(
    count_statistic.TS_null,
    xmin=xmin,
    xmax=0,
    linestyle="dotted",
    color="k",
    label="Null hypothesis",
)
plt.vlines(
    0, ymin=ymin, ymax=count_statistic.TS_null, linestyle="dotted", color="k"
)

plt.vlines(
    excess,
    ymin=count_statistic.TS_max,
    ymax=count_statistic.TS_null,
    color="r",
)
plt.hlines(
    count_statistic.TS_null, xmin=0, xmax=excess, linestyle="dotted", color="r"
)
plt.legend()

### Estimating the confidence range and upper limit of the signal

The 68% confidence interval (or 1 sigma asymmetric errors) is obtained by finding the signal values for which the TS variation is 1. 

In [ ]:
errn = count_statistic.compute_errn(1.0)
errp = count_statistic.compute_errp(1.0)
print(f"68% confidence range: {excess+errn} < mu < {excess+errp}")

The 95% confidence interval (or 2 sigma asymmetric errors) is obtained by finding the signal values for which the TS variation is 4. 

In [ ]:
errn_2sigma = count_statistic.compute_errn(2.0)
errp_2sigma = count_statistic.compute_errp(2.0)
print(
    f"95% confidence range: {excess+errn_2sigma} < mu < {excess+errp_2sigma}"
)

Again for clarity, we plot the statistic profile and the resulting confidence intervals.

In [ ]:
xmin, xmax = -1.5, 25
ymin, ymax = -42, -28.0
plt.figure(figsize=(7, 7))
plt.plot(mu_signal, stat_values, color="k")
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.xlabel(r"Number of expected signal event, $\mu_{sig}$")
plt.ylabel(r"Cash statistic value, TS ")

plt.hlines(
    count_statistic.TS_max + 1,
    xmin=excess + errn,
    xmax=excess + errp,
    linestyle="dotted",
    color="r",
    label="1 sigma (68% C.L.)",
)
plt.vlines(
    excess + errn,
    ymin=ymin,
    ymax=count_statistic.TS_max + 1,
    linestyle="dotted",
    color="r",
)
plt.vlines(
    excess + errp,
    ymin=ymin,
    ymax=count_statistic.TS_max + 1,
    linestyle="dotted",
    color="r",
)

plt.hlines(
    count_statistic.TS_max + 4,
    xmin=excess + errn_2sigma,
    xmax=excess + errp_2sigma,
    linestyle="dashed",
    color="b",
    label="2 sigma (95% C.L.)",
)
plt.vlines(
    excess + errn_2sigma,
    ymin=ymin,
    ymax=count_statistic.TS_max + 4,
    linestyle="dashed",
    color="b",
)
plt.vlines(
    excess + errp_2sigma,
    ymin=ymin,
    ymax=count_statistic.TS_max + 4,
    linestyle="dashed",
    color="b",
)


plt.legend()

## Poisson signal with unknow background: the WStat

We now study the case of an observation yielding $n_{ON}=13$ events, but this time without a model of theexpected number of background events. Instead, a second observation in which we expect twice the number of background has measured $n_{OFF}=11$ events. 

The estimated background in the $ON$ observation is therefore:
$n_{ON} - \alpha n_{OFF} = 5.5$, with $\alpha = 1/2$.

We are dealing with WStat and create a `WStatCountsStatistic` object.

In [ ]:
count_statistic = WStatCountsStatistic(n_on=13, n_off=11, alpha=0.5)

Because of the uncertain $OFF$ measurement, the measured variance is larger than in known background situation

In [ ]:
excess = count_statistic.excess
error = count_statistic.error
print(f"Measured excess : {excess} +- {error}")

### Estimating excess significance

Again, the significance of the measured excess can be estimated by comparing the statistic value for null hypothesis and best fit value, the delta TS, which is distributed as chi2 with 1 d.o.f.

In [ ]:
print(f"Delta TS : {count_statistic.delta_ts}")

We can have a estimate of the significance, expressed in number of sigma, by taking the square root of delta_ts. For WStat, this is exactly equal to the so-called Li&Ma significance. 

In [ ]:
print(f"Square root TS (Li&Ma significance): {count_statistic.significance}")

And we can compute the corresponding p-value: 

In [ ]:
print(f"p_value: {count_statistic.p_value}")

Again, we show how this works with a plot of the WStat profile.

In [ ]:
# We compute the WStat statistic profile
mu_signal = np.linspace(-2.5, 26, 100)
stat_values = count_statistic._stat_fcn(mu_signal)

In [ ]:
xmin, xmax = -2.5, 26
ymin, ymax = 0, 15
plt.figure(figsize=(7, 7))
plt.plot(mu_signal, stat_values, color="k")
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)

plt.xlabel(r"Number of expected signal event, $\mu_{sig}$")
plt.ylabel(r"WStat value, TS ")
plt.vlines(
    excess,
    ymin=ymin,
    ymax=count_statistic.TS_max,
    linestyle="dashed",
    color="k",
    label="Best fit",
)
plt.hlines(
    count_statistic.TS_max,
    xmin=xmin,
    xmax=excess,
    linestyle="dashed",
    color="k",
)
plt.hlines(
    count_statistic.TS_null,
    xmin=xmin,
    xmax=0,
    linestyle="dotted",
    color="k",
    label="Null hypothesis",
)
plt.vlines(
    0, ymin=ymin, ymax=count_statistic.TS_null, linestyle="dotted", color="k"
)

plt.vlines(
    excess,
    ymin=count_statistic.TS_max,
    ymax=count_statistic.TS_null,
    color="r",
)
plt.hlines(
    count_statistic.TS_null, xmin=0, xmax=excess, linestyle="dotted", color="r"
)
plt.legend()

### Estimating the confidence range and upper limit of the signal

Estimation of the confidence range is performed exactly the same way.

In [ ]:
errn = count_statistic.compute_errn(1.0)
errp = count_statistic.compute_errp(1.0)
print(f"68% confidence range: {excess+errn} < mu < {excess+errp}")

In [ ]:
errn_2sigma = count_statistic.compute_errn(2.0)
errp_2sigma = count_statistic.compute_errp(2.0)
print(
    f"95% confidence range: {excess+errn_2sigma} < mu < {excess+errp_2sigma}"
)

Again for clarity, we plot the statistic profile and the resulting confidence intervals.

In [ ]:
xmin, xmax = -2.5, 26
ymin, ymax = 0, 15
plt.figure(figsize=(7, 7))
plt.plot(mu_signal, stat_values, color="k")
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)

plt.xlabel(r"Number of expected signal event, $\mu_{sig}$")
plt.ylabel(r"WStat value, TS ")

plt.hlines(
    count_statistic.TS_max + 1,
    xmin=excess + errn,
    xmax=excess + errp,
    linestyle="dotted",
    color="r",
    label="1 sigma (68% C.L.)",
)
plt.vlines(
    excess + errn,
    ymin=ymin,
    ymax=count_statistic.TS_max + 1,
    linestyle="dotted",
    color="r",
)
plt.vlines(
    excess + errp,
    ymin=ymin,
    ymax=count_statistic.TS_max + 1,
    linestyle="dotted",
    color="r",
)

plt.hlines(
    count_statistic.TS_max + 4,
    xmin=excess + errn_2sigma,
    xmax=excess + errp_2sigma,
    linestyle="dashed",
    color="b",
    label="2 sigma (95% C.L.)",
)
plt.vlines(
    excess + errn_2sigma,
    ymin=ymin,
    ymax=count_statistic.TS_max + 4,
    linestyle="dashed",
    color="b",
)
plt.vlines(
    excess + errp_2sigma,
    ymin=ymin,
    ymax=count_statistic.TS_max + 4,
    linestyle="dashed",
    color="b",
)

plt.legend()